In [56]:
import xml.etree.ElementTree as ET

In [1]:
import os

In [2]:
import zipfile

In [2]:
file_path = "/mnt/c/Users/HP/Documents/drug_labels/"
name = "dm_spl_release_human_rx_part"
extraction_path = "/mnt/c/Users/HP/Documents/drug_labels/extracted"

for i in range(1,5):
    with zipfile.ZipFile(os.path.join(file_path,name+str(i)+".zip"), 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

In [12]:
def data_clean(xml_file_path, output_file_path):
    
    file_path = xml_file_path

    output_file_path_bak = output_file_path+'.bak'
    tree = ET.parse(file_path)

    root = tree.getroot()

    prefix = root.tag.split('}')[0]+'}'

    title_list = []
    string_list = []

    title_list.append('>'+"Name")
    string_list.append([])
    for ats in root[-1][-1][0][0].iter(prefix+'manufacturedProduct'):
        for child in ats[0]:
            if child.tag.endswith('name'):
                string_list[-1].append(child.text.strip())


    title_list.append('>'+"genericMedicine")
    string_list.append([])
    for ats in root[-1][-1][0][0].iter(prefix+"genericMedicine"):
        for child in ats:
            if child.tag.endswith('name'):
                string_list[-1].append(child.text.strip())


    title_list.append('>'+"activeMoiety")
    string_list.append([])
    for ats in root[-1][-1][0][0].iter(prefix+'activeMoiety'):
        for child in ats:
            if child.tag[-4:] == 'name':
                string_list[-1].append(child.text.strip())


    title_list.append('>'+"IngredientSubstance")
    string_list.append([])
    for ats in root[-1][-1][0][0].iter(prefix+'manufacturedProduct'):
        for igs in ats.iter():
            if igs.tag.endswith('Substance'):
                for child in igs.iter():
                    if child.tag.endswith('name'):
                        string_list[-1].append(child.text.strip())

    parent_map = {c: p for p in tree.iter() for c in p}


    for i,child in enumerate(root[-1][-1]):
        if i == 0:
            continue
        title_node = child[0].find(prefix+"title")
        title_list.append('>')
        try:
            for title_text in title_node.itertext():
                title_list[-1] = title_list[-1]+title_text.strip()
        except AttributeError:
            pass


        string_list.append([])
        for txt in child.itertext():
            string_list[-1].append(txt)

    for i in range(4):
        string_list[i] = [_ for _ in set(string_list[i])]

    for j in range(4,len(string_list)):
        k=0
        while (k < len(string_list[j])):
            if string_list[j][k].strip() == '':
                string_list[j][k] = '\n'
                for k2 in range(k+1,len(string_list[j])):
                    if string_list[j][k2].strip() == '':
                        string_list[j][k2] = ''
                        if k2 == len(string_list[j]) -1:
                            k = k2+1
                    else:
                        k = k2
                        break
                continue
            else:
                string_list[j][k] = string_list[j][k].strip().replace('\xa0','')
                k +=1
    for i in range(4,len(string_list)):
        string_list[i][0] = string_list[i][0].strip()


    with open(output_file_path_bak, 'w') as f:
        for i in range(len(title_list)):
            if i < 4:
                f.write(title_list[i]+'\n')
                f.write(" ".join(string_list[i])+'\n')
                continue
            f.write(title_list[i]+'\n')
            f.write("".join(string_list[i])+'\n')
    
    with open(output_file_path,'w') as f1, open(output_file_path_bak,'r') as f2:
        for line in f2:
            if line.strip() == '':
                continue
            f1.write(line.strip()+'\n')
    os.remove(output_file_path_bak)
    return string_list[0]

In [13]:
def file_extraction(file_path, output_dir):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

In [3]:
def temp_dir_remove(dir_path, include_base_dir = False):
    # Delete everything reachable from the directory named in 'top',
    # assuming there are no symbolic links.
    # CAUTION:  This is dangerous!  For example, if top == '/', it
    # could delete all your disk files.
    top = dir_path
    for root, dirs, files in os.walk(top, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    if include_base_dir == True:
        os.rmdir(dir_path)

In [4]:
import pickle
def save_obj(obj,path,name):
    with open(os.path.join(path, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [5]:
output_file_dir = "/mnt/c/Users/HP/Documents/drug_labels/cleanup"
temp_dir = "/mnt/c/Users/HP/Documents/drug_labels/tmp"

In [7]:
directory_in_str = "/mnt/c/Users/HP/Documents/drug_labels/extracted/prescription"

In [26]:
counter = 0
name2id = {}
id2name = {}

directory = os.fsencode(directory_in_str)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".zip") : 
        os.mkdir(temp_dir)
        file_extraction(os.path.join(directory_in_str, filename),temp_dir)
        for subdir, dirs, files in os.walk(temp_dir):
            for file in files:
                if file.endswith(".xml"):
                    name = data_clean(os.path.join(subdir,file), os.path.join(output_file_dir,str(counter)+'.txt'))
                    id2name[counter] = name
                    for n in name:
                        name2id[n] = counter
                    counter +=1
        temp_dir_remove(temp_dir,include_base_dir=True)

In [38]:
save_obj(id2name, file_path,"id2name")

In [39]:
save_obj(name2id, file_path,"name2id")

In [70]:
id2name_no_redundancy = {}

for item in name2id.items() :
    if item[1] in id2name_no_redundancy:
        id2name_no_redundancy[item[1]].append(item[0])
    else:
        id2name_no_redundancy[item[1]] = [item[0]]

In [71]:
len(id2name_no_redundancy)

7977

In [72]:
save_obj(id2name_no_redundancy, file_path,"id2name_no_redundancy")

In [73]:
id2name_no_redundancy

{0: ['Renese'],
 1: ['Mykrox Tablets'],
 2: ['Tolinase'],
 3: ['HYPAQUE'],
 28223: ['Meperidine Hydrochloride'],
 5: ['Halotestin'],
 164: ['Hypaque Sodium'],
 9912: ['Dantrium'],
 8: ['Lithane'],
 9: ['Renese-R'],
 10: ['Trovan'],
 11: ['Plasma-Lyte R'],
 317: ['Psorcon'],
 14: ['Maxaquin'],
 15: ['Terramycin'],
 36946: ['Cortisone Acetate'],
 17: ['Plasma-Lyte 148 and dextrose'],
 42845: ['Atenolol'],
 19: ['Enlon-Plus'],
 20: ['Urobiotic'],
 42557: ['Hydrocodone Bitartrate and Acetaminophen'],
 4811: ['Scandonest L'],
 4813: ['Scandonest Plain'],
 23: ['Cimetidine in Sodium Chloride'],
 15036: ['Gastrocrom'],
 25: ['Terra-Cortril'],
 26: ['Verapamil HCl'],
 27: ['Florone'],
 40736: ['Meclizine Hydrochloride'],
 40942: ['Metoprolol tartrate'],
 42486: ['Lorazepam'],
 42068: ['Oxybutynin Chloride'],
 160: ['Psorcon E'],
 33: ['Fluor-Op'],
 34: ['Chlordiazepoxide HCl'],
 35: ['Livostin'],
 25099: ['Targretin'],
 37: ['HMS'],
 42146: ['Metronidazole'],
 5884: ['Promethazine HCl'],
 159:

os.mkdir(os.path.join(file_path,"cleanup_no_redun"))

In [6]:
import shutil

for key in id2name_no_redundancy.keys():
    shutil.copyfile(os.path.join(output_file_dir, str(key)+'.txt'), os.path.join(file_path,"cleanup_no_redun",str(key)+'.txt'))

In [55]:
dir_in_str = os.path.join(file_path,"cleanup_no_redun")

directory = os.fsencode(dir_in_str)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".txt") : 
        os.mkdir(temp_dir)
        with open(os.path.join(dir_in_str,filename), 'r') as f1, open(os.path.join(temp_dir,filename), 'w') as f2:
            for line in f1:
                if line.rfind('[') > line.rfind(']'):
                    f2.write(line.strip('\n'))
                    while True: # for python 3.8 text := f1.readline()
                        text = f1.readline()
                        if not text:
                            break
                        if text.find(']') != -1:
                            f2.write(text)
                            break
                        else:
                            f2.write(text.strip('\n'))
                elif line.rfind('(') > line.rfind(')'):
                    f2.write(line.strip('\n'))
                    while True:
                        text = f1.readline()
                        if not text:
                            break
                        if text.find(')') != -1:
                            f2.write(text)
                            break
                        else:
                            f2.write(text.strip('\n'))
                else:
                    f2.write(line)
        os.remove(os.path.join(dir_in_str,filename))
        shutil.copyfile(os.path.join(temp_dir,filename),os.path.join(dir_in_str,filename))                       
        temp_dir_remove(temp_dir,include_base_dir=True)

In [11]:
id2name_no_redundancy = load_obj(os.path.join(file_path,"id2name_no_redundancy.pkl"))

In [91]:
name2id = load_obj(os.path.join(file_path,"name2id.pkl"))

In [93]:
id2name =load_obj(os.path.join(file_path,"id2name.pkl"))

In [94]:
 len([_ for _ in id2name.keys()])

42850

In [19]:
id2name_no_redundancy[11400]

['Dibasic Sodium Phosphate, Monobasic Potassium Phosphate and Monobasic Sodium Phosphate']

In [57]:
file_path_db = "/mnt/c/Users/HP/Documents/drug_labels/drugbank_all_full_database/full_database.xml"

In [58]:
tree = ET.parse(file_path_db)
root = tree.getroot()

In [60]:
len(root)

14315

In [70]:
for child in root[1000]:
    print(child.tag, child.attrib)

{http://www.drugbank.ca}drugbank-id {'primary': 'true'}
{http://www.drugbank.ca}drugbank-id {}
{http://www.drugbank.ca}name {}
{http://www.drugbank.ca}description {}
{http://www.drugbank.ca}cas-number {}
{http://www.drugbank.ca}unii {}
{http://www.drugbank.ca}average-mass {}
{http://www.drugbank.ca}monoisotopic-mass {}
{http://www.drugbank.ca}state {}
{http://www.drugbank.ca}groups {}
{http://www.drugbank.ca}general-references {}
{http://www.drugbank.ca}synthesis-reference {}
{http://www.drugbank.ca}indication {}
{http://www.drugbank.ca}pharmacodynamics {}
{http://www.drugbank.ca}mechanism-of-action {}
{http://www.drugbank.ca}toxicity {}
{http://www.drugbank.ca}metabolism {}
{http://www.drugbank.ca}absorption {}
{http://www.drugbank.ca}half-life {}
{http://www.drugbank.ca}protein-binding {}
{http://www.drugbank.ca}route-of-elimination {}
{http://www.drugbank.ca}volume-of-distribution {}
{http://www.drugbank.ca}clearance {}
{http://www.drugbank.ca}classification {}
{http://www.drugbank.

In [78]:
root[1000].find("{http://www.drugbank.ca}name").text

'Guanfacine'

In [80]:
root[1000][0].text

'DB01018'

In [85]:
DBID2name = {}
name2DBID = {}

for i,child in enumerate(root):
    
    name = child.find("{http://www.drugbank.ca}name").text
    DBID2name[child[0].text] = name
    name2DBID[name] = child[0].text

In [88]:
save_obj(DBID2name,"/mnt/c/Users/HP/Documents/drug_labels/","DBID2name")
save_obj(name2DBID,"/mnt/c/Users/HP/Documents/drug_labels/","name2DBID")

In [106]:
name2id_lower = {}
for item in name2id.items():
    name2id_lower[item[0].lower()] = item[1]

In [108]:
save_obj(name2id_lower,"/mnt/c/Users/HP/Documents/drug_labels/","name2id_lower")

In [109]:
id2name_no_redundancy_lower = {}
for item in id2name_no_redundancy.items():
    id2name_no_redundancy_lower[item[0]] = [_.lower() for _ in item[1]]

In [111]:
save_obj(id2name_no_redundancy_lower,"/mnt/c/Users/HP/Documents/drug_labels/","id2name_no_redundancy_lower")

In [112]:
DBID2name_lower = {}

for item in DBID2name.items():
    DBID2name_lower[item[0]] = item[1].lower()
    
save_obj(DBID2name_lower,"/mnt/c/Users/HP/Documents/drug_labels/","DBID2name_lower")

In [113]:
name2DBID_lower = {}

for item in name2DBID.items():
    name2DBID_lower[item[0].lower()] = item[1]
    
save_obj(name2DBID_lower,"/mnt/c/Users/HP/Documents/drug_labels/","name2DBID_lower")

In [95]:
ddi_dual_dict =load_obj(os.path.join(file_path,"ddi_dual_dict.pkl"))

In [116]:
existed_list = []
dbid_list = []
id_list = []
for key in ddi_dual_dict.keys():
    if key in DBID2name_lower:
        name = DBID2name_lower[key]
        if name in name2id_lower:
            existed_list.append(name)
            dbid_list.append(key)
            id_list.append(name2id_lower[name])

In [119]:
len(id_list)

576

In [121]:
DDI_pair_list = []
for i in range(len(dbid_list)):
    for j in range(i,len(dbid_list)):
        if dbid_list[j] in ddi_dual_dict[dbid_list[i]]:
            DDI_pair_list.append([dbid_list[i],dbid_list[j]])

In [123]:
len(DDI_pair_list)

32192

In [126]:
save_obj(DDI_pair_list,file_path,"ddi_test_set_dbid")

In [127]:
id_pair_list = []

for pair in DDI_pair_list:
    id_pair_list.append([ name2id_lower[DBID2name_lower[pair[0]]] , name2id_lower[DBID2name_lower[pair[1]]] ])

In [129]:
save_obj(id_pair_list,file_path,"ddi_test_set_id")